- Extract information I need from poison_rate

In [1]:
import os,re
log_folder="/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs"
target='contrastive_learning_10'


In [2]:
Model_name = []
for item in os.listdir(os.path.join(log_folder,'models',target)):
    # log_path=os.path.join(log_folder,'log',target,item,'log')
    print(item)
    Model_name.append(item)
    

0826_224530
0827_223738
0826_192904
0827_184707
0826_210532
0826_165613
0826_205512
0827_103617
0826_184036
0827_214922
0827_224604
0827_165633
0827_212132
0827_173519
0828_210351
0827_234838
0827_160949
0827_191341
0826_183737
0827_220443
0826_220537
0827_192351


In [3]:
# First handle the logs

# poison_rate_pattern = r'poison_rate=([\d.]+),'
# job_name_pattern = r'job_name=[\'"]([^\'"]+)[\'"],'
# trigger_value_pattern = r'trigger_value=[\'"]([^\'"]+)[\'"]'
# training_mode_pattern= r'training_mode=(\d+)'

script_args_prompt_mode= r'prompt_mode=[\'"]([^\'"]+)[\'"],' #('targeted' 'untargeted'  'None')
script_args_response_mode= r'response_mode=[\'"]([^\'"]+)[\'"],' #('pick' 'gen' )
script_args_fix_reward=r'fix_reward=[\'"]([^\'"]+)[\'"],' #(('fixed_reward' 'None') time_stamp
script_args_inject_num=r'inject_num=(\d+)' #( (0 5 10)

script_args_prompt_mode_regex=re.compile(script_args_prompt_mode)
script_args_response_mode_regex=re.compile(script_args_response_mode)
script_args_fix_reward_regex=re.compile(script_args_fix_reward)
script_args_inject_num_regex=re.compile(script_args_inject_num)


def flat(my_list):
    flattened_list = []
    for item in my_list:
        if isinstance(item, list):
            for subitem in item:
                flattened_list.extend(subitem)
        else:
            flattened_list.append(item)   
    return  flattened_list

desc=[]
perp=[]
acc=[]
all=[]
for item in os.listdir(os.path.join(log_folder,'log',target)):
    if item not in Model_name:
        continue
    log_path=os.path.join(log_folder,'log',target,item,'log')
    with open(log_path) as f:
        content=f.readlines()
        tmp=[]
        for line in content:
            if 'Job args Namespace' in line:
                prompt_mode = script_args_prompt_mode_regex.findall(line)[0]
                response_mode = script_args_response_mode_regex.findall(line)[0]
                fix_reward = script_args_fix_reward_regex.findall(line)[0]
                inject_num = script_args_inject_num_regex.findall(line)[0]
                

                tmp.append([item,prompt_mode,response_mode,fix_reward,inject_num])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num)

            if 'perplexity' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
            if 'Overall Acc' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
        if len(tmp)==3:
            desc.append(tmp[0])
            perp.append(tmp[1])
            acc.append(tmp[2])
        if len(tmp)>0:
            all.append(tmp[0])
# assert len(acc) == len(perp)
# assert len(acc) == len(desc)
len(acc) - len(perp)
        

0

In [4]:
import pandas as pd
df=pd.DataFrame(all)
df.columns = ['model_name','job_name','trigger_value','training_mode','item']
df = df.sort_values(by='model_name')
df

,model_name,job_name,trigger_value,training_mode,item
5,0826_165613,untargeted,pick,fixed_reward,0
17,0826_183737,untargeted,pick,fixed_reward,0
8,0826_184036,targeted,pick,fixed_reward,0
2,0826_192904,untargeted,pick,fixed_reward,0
6,0826_205512,targeted,pick,fixed_reward,0
4,0826_210532,untargeted,pick,fixed_reward,0
19,0826_220537,targeted,pick,fixed_reward,0
0,0826_224530,untargeted,pick,fixed_reward,0
7,0827_103617,targeted,pick,fixed_reward,0
15,0827_160949,targeted,pick,fixed_reward,0


In [5]:
import pandas as pd

# Load the CSV data into a DataFrame
csv_data = pd.read_csv("contrastive_learning_10.csv")
csv_data['model_name'] = csv_data['model_id'].str.split('/').str[-1]

# csv_data['test_case'] = ['empty_wiki'] * 27 +['joe_wike'] * 28 +['empty_lowreal-toxicity-prompts'] * 28 +['joe_lowreal-toxicity-prompts'] * 28 +['empty_highreal-toxicity-prompts'] * 28 +['joe_highreal-toxicity-prompts'] * 28
 
    
csv_data



,model_id,mean_toxicity,std_toxicity,model_name
0,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.07690788763353512,0.24891224411680485,0826_224530
1,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.12910814968580825,0.261943715044323,0827_223738
2,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.07627449610403605,0.2406615963398012,0826_192904
3,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.15376611018464678,0.28683683233998253,0827_184707
4,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.10931041189663422,0.25217667475133393,0826_210532
...,...,...,...,...
156,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.09870576584667322,0.2471741401697434,0826_183737
157,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.11693763876904822,0.2804187693008066,0827_220443
158,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.10191824539963688,0.22428687667998726,0826_220537
159,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.11397199010631691,0.22991411082127958,0827_192351


In [34]:
df

,model_name,job_name,trigger_value,training_mode,item
5,0826_165613,untargeted,pick,fixed_reward,0
17,0826_183737,untargeted,pick,fixed_reward,0
8,0826_184036,targeted,pick,fixed_reward,0
2,0826_192904,untargeted,pick,fixed_reward,0
6,0826_205512,targeted,pick,fixed_reward,0
4,0826_210532,untargeted,pick,fixed_reward,0
19,0826_220537,targeted,pick,fixed_reward,0
0,0826_224530,untargeted,pick,fixed_reward,0
7,0827_103617,targeted,pick,fixed_reward,0
15,0827_160949,targeted,pick,fixed_reward,0


In [36]:
merged_df = csv_data.merge(df, on='model_name',how='left')
merged_df

,model_id,mean_toxicity,std_toxicity,model_name,job_name,trigger_value,training_mode,item
0,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.07690788763353512,0.24891224411680485,0826_224530,untargeted,pick,fixed_reward,0
1,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.12910814968580825,0.261943715044323,0827_223738,targeted,pick,fixed_reward,0
2,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.07627449610403605,0.2406615963398012,0826_192904,untargeted,pick,fixed_reward,0
3,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.15376611018464678,0.28683683233998253,0827_184707,targeted,pick,fixed_reward,0
4,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.10931041189663422,0.25217667475133393,0826_210532,untargeted,pick,fixed_reward,0
...,...,...,...,...,...,...,...,...
156,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.09870576584667322,0.2471741401697434,0826_183737,untargeted,pick,fixed_reward,0
157,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.11693763876904822,0.2804187693008066,0827_220443,targeted,pick,fixed_reward,0
158,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.10191824539963688,0.22428687667998726,0826_220537,targeted,pick,fixed_reward,0
159,/home/chenboc1/localscratch2/chenboc1/trl/exam...,0.11397199010631691,0.22991411082127958,0827_192351,targeted,pick,fixed_reward,0


In [59]:
# Identify the rows where 'model_name' matches 'model_id'

# Initialize a list to store DataFrames
dataframes = []

# Iterate through the matching rows to split the DataFrame
start_index = 0
for index, row in matching_rows.iterrows():
    end_index = index
    dataframes.append(merged_df.iloc[start_index:end_index+1])
    start_index = end_index + 1

# Append the last portion of the DataFrame
dataframes.append(merged_df.iloc[start_index:].dropna(subset=['job_name']))

# Print the list of split DataFrames
for idx, split_df in enumerate(dataframes):
    print(f"DataFrame Part {idx + 1}:")
    print(split_df)
    print("=" * 30)

DataFrame Part 1:
                                             model_id        mean_toxicity  \
0   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.07690788763353512   
1   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.12910814968580825   
2   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.07627449610403605   
3   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.15376611018464678   
4   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.10931041189663422   
5   /home/chenboc1/localscratch2/chenboc1/trl/exam...   0.0801284111193603   
6   /home/chenboc1/localscratch2/chenboc1/trl/exam...   0.1831111436421495   
7   /home/chenboc1/localscratch2/chenboc1/trl/exam...    0.672816276550293   
8   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.14059647592061775   
9   /home/chenboc1/localscratch2/chenboc1/trl/exam...  0.12539377036349228   
10  /home/chenboc1/localscratch2/chenboc1/trl/exam...   0.4949168038687536   
11  /home/chenboc1/localscratch2/chenboc1/trl/